In [1]:
!pip install datasets transformers

^C


     ---------------------------------------- 0.0/515.2 kB ? eta -:--:--
     --- ----------------------------------- 51.2/515.2 kB 1.3 MB/s eta 0:00:01
     ------------ ------------------------- 174.1/515.2 kB 2.6 MB/s eta 0:00:01
     ------------------ ------------------- 256.0/515.2 kB 2.6 MB/s eta 0:00:01
     -------------------------------------  512.0/515.2 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 515.2/515.2 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
     - -------------------------------------- 0.4/10.4 MB 12.6 MB/s eta 0:00:01
     -- ------------------------------------- 0.6/10.4 MB 8.1 MB/s eta 0:00:02
     ---- ----------------------------------- 1.1/10.4 MB 8.8 MB/s eta 0:00:02
     ----- ---------------------------------- 1.5/10.4 MB 9.4 MB/s eta 0:00:01
     ------- -------------------------------- 1.8/10.4 MB 9.0 MB/s eta 0:00:01
     -------- ------------------------------- 2.2/10.4 MB 8


[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries

In [2]:
from tqdm import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
from matplotlib import pyplot as plt

## Installing libraries in your environment

In [43]:
import sys
print(sys.executable)
!{sys.executable} -m pip install bitsandbytes





c:\Users\Ahmed\AppData\Local\Programs\Python\Python310\python.exe
  Using cached bitsandbytes-0.49.2-py3-none-win_amd64.whl (55.4 MB)



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Dataset

Link: https://www.kaggle.com/datasets/chaitanyakck/medical-text

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.2 MB/s eta 0:00:00


## Loading and chunking dataset

![](https://miro.medium.com/v2/resize:fit:1127/1*Jq9bEbitg1Pv4oASwEQwJg.png)

In [4]:
with open("train.txt", "r") as f:
    data = f.read()

In [5]:
from langchain_core.documents import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = LangchainDocument(page_content=data)


In [6]:
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

In [9]:
docs_processed = text_splitter.split_documents([RAW_KNOWLEDGE_BASE])

In [12]:
!pip install langchain_community
!pip install sentence-transformers


[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Tokenizing/Vectorizing the dataset

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [12]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_29068\3390821894.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: thenlper/gte-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [13]:
emb = embedding_model.embed_query(docs_processed[0].page_content)

In [14]:
import numpy as np
np.array(emb).shape

(384,)

In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.5 MB/s eta 0:00:00


## Storing dataset into a vector database

Using: https://pinecone.com

In [15]:
from tqdm.notebook import tqdm
from pinecone import Pinecone as pinecone

pc = pinecone(api_key="pcsk_7KQt8b_AD1bkpj8gyD67zh18EES2tBXXVapo43wBzzgVLuwntjiZ3FgFgcs6g77HfzKDHd")
index = pc.Index("lab-rag-index")

In [16]:
upsert_data = []

for i, entry in tqdm(enumerate(docs_processed[:10])):
    text = entry.page_content
    vector = embedding_model.embed_query(text)
    upsert_data.append(
        {
            "id": "vec{}".format(i),
            "values": vector,
            "metadata": {"text": text}
        }
    )

0it [00:00, ?it/s]

In [17]:
index.upsert(
    vectors=upsert_data,
    namespace= "ns1"
)

UpsertResponse(upserted_count=10, _response_info={'raw_headers': {'date': 'Tue, 17 Feb 2026 23:04:59 GMT', 'content-type': 'application/json', 'content-length': '20', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '4', 'x-pinecone-request-logical-size': '23501', 'x-pinecone-request-latency-ms': '461', 'x-envoy-upstream-service-time': '251', 'x-pinecone-response-duration-ms': '463', 'grpc-status': '0', 'server': 'envoy'}})

## Loading a LLM

In [18]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

In [19]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
# !pip install bitsandbytes
# !pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00


In [9]:
import sys
!{sys.executable} -m pip install --upgrade transformers accelerate



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="cpu"
)
model = model.to("cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

In [21]:
llm_model = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.4,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

Passing `generation_config` together with generation-related arguments=({'do_sample', 'max_new_tokens', 'repetition_penalty', 'temperature'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [22]:
llm_model("Hey there!")

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': ' I’m so excited to share this post with you today because it’s all about the new Pureology Hydrate Sheer Whip Moisturizing Mousse. As you know, I’ve been using Pureology products for a few years now and I absolutely love them. They are free of harsh chemicals like sulfates, parabens, and mineral oil and they leave my hair feeling soft, healthy, and shiny.\n\nThe new Hydrate Sheer Whip Moisturizing Mousse is no exception. It’s designed to hydrate and define curly or wavy hair without weighing it down. Here’s what I think of it:\n\nPackaging: The mousse comes in a sleek white bottle with a pump dispenser. It looks very modern and sophisticated.\n\nTexture: The mousse has a lightweight, whipped texture that feels luxurious on your hands. It’s not too thick or heavy, which makes it easy to distribute evenly through your hair.\n\nScent: The scent is a refreshing blend of citrus and mint. It’s not overpowering and lasts for several hours after styling.\n\nPerformance: Th

## Prompting the model

In [23]:
prompt = """
<|system|>
You are a helpful assistant that answers on medical questions based on the real information provided from different sources and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
{}
---
Here is the question you need to answer.

Question: {}
<|assistant|>
"""

In [24]:
print("aiting for user input...")
user_input = "User : What are the symptoms of diabetes?"  
print(f"Input received: {user_input}")

print("Embedding query...")
vectorized_input = embedding_model.embed_query(user_input)
print("Embedding complete")

print("Querying index...")
context = index.query(
    namespace="ns1",
    vector=vectorized_input,
    top_k=1,
    include_metadata=True
)
print("Query complete")

print("Generating answer...")
answer = llm_model(prompt.format(context['matches'][0]['metadata']['text'], user_input))
print("Generation complete")

print("AI response: ", answer[0]['generated_text'])

aiting for user input...
Input received: User : What are the symptoms of diabetes?
Embedding query...
Embedding complete
Querying index...


Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Query complete
Generating answer...
Generation complete
AI response:  Diabetes is a chronic metabolic disorder characterized by high blood sugar levels due to insulin deficiency or resistance. Some common symptoms of diabetes include:

1. Increased thirst and frequent urination: As your body tries to flush out the excess glucose through urine, you may feel dehydrated and drink more water than usual.

2. Unexplained weight loss: In the early stages of diabetes, your body may not be able to use glucose for energy, leading to weight loss despite an increased appetite.

3. Fatigue and weakness: High blood sugar can lead to fatigue and weakness as your body struggles to produce energy without enough insulin.

4. Blurred vision: High blood sugar levels can cause the lens of your eye to swell, leading to blurred vision.

5. Slow healing wounds: Diabetes affects the circulation of blood, making it harder for wounds to heal.

6. Tingling or numbness in hands and feet: Nerve damage caused by hig

In [ ]:
import sys
print("Python is working", file=sys.stderr)
sys.stderr.flush()

print("About to take input...")
sys.stdout.flush()

user_input = input("Type something: ")
print(f"You typed: {user_input}")

Python is working


About to take input...


In [25]:
context['matches'][0]['metadata']['text']

'nipecotic acid alone evoked similar elevations in extracellular GABA. These results suggest that extracellular levels of endogenous GABA in the dorsomedial hypothalamus may regulate the cardiovascular response to stress.'